In [8]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from numpy import random

import pandas as pd
import seaborn as sns

In [9]:
plotly.tools.set_credentials_file(username='asura.enkhbayar', api_key='eS08y0AycFE6HC4EjBm2')

In [12]:
data = pd.read_csv("dummy.csv", dtype={'Grant duration (months)':str})
data.head()

,Organisation,Grant name (leave empty if no grant),Grant size or range (USD),Grant duration (years),Type,Link
0,Mozilla,Mini-Grants,2-1100,0.5,Private NGO,https://www.opportunitiesforafricans.com/mozil...
1,Sloan,NaN,50,0.5,Private NGO,NaN
2,MOSS,NaN,10-100,0.5,Private NGO,NaN
3,eLife,Innovation Initiative,10-50,1,Corporate,https://elifesciences.org/about/innovation
4,Welcome Trust,NaN,50,1,Corporate,NaN


In [13]:
money_cats = [0, 25, 100, 250, 1000, 5000]
time_cats = [0, 1, 3, 5]

def categorise(rows, categories):
    output = []
    for row in rows:
        min_g, max_g = None, None
        if "-" in row:
            values = [float(g) for g in row.split("-")]
        else:
            values = [float(row), float(row)]

        for c in categories:
            if c >= values[0] and not min_g:
                min_g = categories.index(c)-1
            if c >= values[1] and not max_g:
                max_g = categories.index(c)
                break
        output.append(categories[min_g:max_g])
    return output

data['grant'] = categorise(data['Grant size or range (USD)'], money_cats)
data['duration'] = categorise(data['Grant duration (years)'], time_cats)
data['label'] = data.apply(lambda x: " - ".join([s for s in x[['Organisation', 'Grant name (leave empty if no grant)']] if pd.notnull(s)]), axis=1)
data

,Organisation,Grant name (leave empty if no grant),Grant size or range (USD),Grant duration (years),Type,Link,grant,duration,label
0,Mozilla,Mini-Grants,2-1100,0.5,Private NGO,https://www.opportunitiesforafricans.com/mozil...,"[25, 100, 250, 1000]",[0],Mozilla - Mini-Grants
1,Sloan,NaN,50,0.5,Private NGO,NaN,[25],[0],Sloan
2,MOSS,NaN,10-100,0.5,Private NGO,NaN,[25],[0],MOSS
3,eLife,Innovation Initiative,10-50,1,Corporate,https://elifesciences.org/about/innovation,[25],[0],eLife - Innovation Initiative
4,Welcome Trust,NaN,50,1,Corporate,NaN,[25],[0],Welcome Trust
5,Sloan,NaN,1250,2-4,Government,NaN,[1000],"[1, 3]",Sloan


In [14]:
types = data.Type.unique().tolist()
traces = []
for c, t in enumerate(types):
    df = data[data.Type == t]
    xs = []
    ys = []
    texts = []
    
    for index, row in df.iterrows():
        for grant_size in row['grant']:
            for grant_duration in row['duration']:
                xs.append(money_cats.index(grant_size)+0.5+random.normal(0, .25))
                ys.append(time_cats.index(grant_duration)+0.5+random.normal(0, .25))
                texts.append(row['label'])
                 
    trace = go.Scatter(
        y = ys,
        x = xs,
        mode = 'markers+text',
        marker= dict(size= 10,
                     line= dict(width=2),
                     color=c,
                     colorscale='Viridis'
                    ),
        text = texts,
        name=t,
        textposition='top center'
    )
    traces.append(trace)

In [15]:
layout= go.Layout(
    title= 'Funding Landscape (months)',
    hovermode= 'closest',
    yaxis= dict(
        title= 'Grant Duration',
    ),
    xaxis=dict(
        title= 'Grant Size (USD)',
    ),
    showlegend= True
)

In [16]:
# Plot and embed in ipython notebook!
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='basic-scatter')